In [1]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [2]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(y_train.shape,  y_test.shape)

(50000, 10) (10000, 10)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=2045)

print(X_train.shape, y_train.shape, '/', X_valid.shape, y_valid.shape, '/', X_test.shape, y_test.shape)

(35000, 32, 32, 3) (35000, 10) / (15000, 32, 32, 3) (15000, 10) / (10000, 32, 32, 3) (10000, 10)


In [4]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top = False, input_shape=(32,32, 3))   ######### input_shape #########
conv_base.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
______________________________________________________________

In [5]:
X_train = X_train.astype(float)/255
X_valid = X_valid.astype(float)/255
X_test = X_test.astype(float)/255

In [ ]:
import numpy as np
features = np.zeros(shape = (35000, 1,1,512))
labels = np.zeros(shape = (35000, 10))

features_batch = conv_base.predict(X_train)

features_batch = features


In [8]:
features_batch.shape

(35000, 1, 1, 512)

In [ ]:
labels = y_train
lables[:1]

In [9]:
def extract_features(x, y, sample_count):
  features = np.zeros(shape =(sample_count, 1, 1, 512))
  labels = np.zeros(shape =(sample_count, 10))

  features_batch = conv_base.predict(x)
  features_batch = features
  labels = y
  return features, labels

In [10]:
X_train, y_train = extract_features(X_train, y_train, 35000)
X_valid, y_valid = extract_features(X_valid, y_valid, 15000)
X_test, y_test = extract_features(X_test, y_test, 10000)

print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

(35000, 1, 1, 512) (35000, 10)
(15000, 1, 1, 512) (15000, 10)
(10000, 1, 1, 512) (10000, 10)


In [11]:
X_train = np.reshape(X_train, (35000, 1*1*512))       ########## 데이터 크기 바꿔주기
X_valid = np.reshape(X_valid, (15000, 1*1*512))
X_test = np.reshape(X_test, (10000, 1*1*512))

X_train.shape, X_valid.shape, X_test.shape

((35000, 512), (15000, 512), (10000, 512))

In [14]:
from keras import models, layers

model = models.Sequential()
model.add(layers.Dense(256, activation ='relu', input_dim=(1*1*512)))  ######### input_dim #################
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax')) ######### sigmoid or softmax ############

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  ############# binary or categorical ########

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 133,898
Trainable params: 133,898
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time
Hist = model.fit(X_train, y_train, epochs=100, batch_size=20, validation_data=(X_valid, y_valid))

Epoch 1/100
1750/1750 [==============================] - 6s 3ms/step - loss: 2.3027 - accuracy: 0.0957 - val_loss: 2.3029 - val_accuracy: 0.0989
Epoch 2/100
1750/1750 [==============================] - 5s 3ms/step - loss: 2.3027 - accuracy: 0.1015 - val_loss: 2.3027 - val_accuracy: 0.0976
Epoch 3/100
1750/1750 [==============================] - 5s 3ms/step - loss: 2.3028 - accuracy: 0.1011 - val_loss: 2.3029 - val_accuracy: 0.0979
Epoch 4/100
1750/1750 [==============================] - 5s 3ms/step - loss: 2.3028 - accuracy: 0.0988 - val_loss: 2.3030 - val_accuracy: 0.0985
Epoch 5/100
1750/1750 [==============================] - 5s 3ms/step - loss: 2.3027 - accuracy: 0.1032 - val_loss: 2.3026 - val_accuracy: 0.0976
Epoch 6/100
1750/1750 [==============================] - 5s 3ms/step - loss: 2.3027 - accuracy: 0.0968 - val_loss: 2.3029 - val_accuracy: 0.0987
Epoch 7/100
1750/1750 [==============================] - 5s 3ms/step - loss: 2.3027 - accuracy: 0.0999 - val_loss: 2.3028 - val_ac